<a href="https://colab.research.google.com/github/pennya6/AI_study/blob/main/computervision/dogandcats_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
import pathlib

data_dir='./dogandcats'
data_dir=pathlib.Path(data_dir)

In [5]:
image_count=len(list(data_dir.glob('*/*.jpg')))
print(image_count)

0
